In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

### Load the data

In [2]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Undersampling by ethnicity

In [3]:
e = data["Ethnicity"]
print('Original dataset shape %s' % Counter(e))

ethnicity_counts = dict(data["Ethnicity"].value_counts())
num_ethnicities = len(ethnicity_counts)
min_count = min(ethnicity_counts.values())

strategy_under = {ethnicity: min_count for ethnicity in range(num_ethnicities)}
under = RandomUnderSampler(sampling_strategy=strategy_under)
print(strategy_under, sep='\n')

data_under, e_under=under.fit_resample(data, e)
print('Undersampled dataset shape %s' % Counter(e_under))

Original dataset shape Counter({0: 1278, 1: 454, 3: 211, 2: 206})
{0: 206, 1: 206, 2: 206, 3: 206}
Undersampled dataset shape Counter({0: 206, 1: 206, 2: 206, 3: 206})


### Split the data

In [4]:
X, y = data_under.drop("Diagnosis", axis=1), data_under["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_train.head()

PatientID  Age  Gender  Ethnicity  EducationLevel        BMI  Smoking  \
448        5199   83       0          2               0  26.837741        0   
444        5195   84       1          2               1  32.396430        0   
1044       5795   90       0          0               2  18.408831        0   
1414       6165   71       1          1               1  23.263395        1   
1848       6599   83       1          2               1  32.153990        0   

      AlcoholConsumption  PhysicalActivity  DietQuality  ...       MMSE  \
448            13.237107          9.071344     7.237438  ...   3.403500   
444            13.942602          4.677394     0.220226  ...   4.929254   
1044            9.844113          5.922700     6.337522  ...  12.769866   
1414           11.785543          2.788285     4.271158  ...   0.455314   
1848            8.697229          1.510410     0.090636  ...  16.893635   

      FunctionalAssessment  MemoryComplaints  BehavioralProblems       ADL  \
448               1.329128                 0                   1  6.743606   
444               2.293248                 1                   1  8.016928   
1044              8.796442                 0                   0  4.032626   
1414              5.464538                 0                   0  7.867315   
1848              9.520252                 1                   0  5.640297   

      Confusion  Disorientation  PersonalityChanges  \
448           0               0                   0   
444           1               1                   0   
1044          1               0                   0   
1414          0               0                   0   
1848          0               0                   0   

      DifficultyCompletingTasks  Forgetfulness  
448                           0              0  
444                           0              0  
1044                          0              0  
1414                          0              0  
1848                          0              1  

[5 rows x 33 columns]

### Train the model

In [5]:
classifier = HistGradientBoostingClassifier(random_state=12)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       135
           1       0.93      0.93      0.93        71

    accuracy                           0.95       206
   macro avg       0.95      0.95      0.95       206
weighted avg       0.95      0.95      0.95       206



In [7]:
X["Ethnicity"].value_counts()

Ethnicity
0    206
1    206
2    206
3    206
Name: count, dtype: int64

### Compare performance based on ethnicity

In [8]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [10]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        31
           1       0.94      1.00      0.97        15

    accuracy                           0.98        46
   macro avg       0.97      0.98      0.98        46
weighted avg       0.98      0.98      0.98        46



In [11]:
# Ethncity 1
eth1_results = results[results["Ethnicity"] == 1]
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        36
           1       0.80      1.00      0.89        12

    accuracy                           0.94        48
   macro avg       0.90      0.96      0.92        48
weighted avg       0.95      0.94      0.94        48



In [12]:
# Ethnicity 2  
eth2_results = results[results["Ethnicity"] == 2]
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94        32
           1       0.95      0.88      0.91        24

    accuracy                           0.93        56
   macro avg       0.93      0.92      0.93        56
weighted avg       0.93      0.93      0.93        56



In [13]:
# Ethncity 3
eth3_results = results[results["Ethnicity"] == 3]
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97        36
           1       1.00      0.90      0.95        20

    accuracy                           0.96        56
   macro avg       0.97      0.95      0.96        56
weighted avg       0.97      0.96      0.96        56



In [25]:
%load_ext autoreload
%autoreload 2
from fairness_metrics import demographic_parity, equalized_odds, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

for i in [1,2,3]:
    dp = round(demographic_parity(results, 0, i), 4)
    eo = equalized_odds(results, 0, i)
    tpr_diff, fpr_diff = round(eo[0], 4), round(eo[1], 4)
    di = round(disparate_impact(results, 0, i), 4)
    print(dp, tpr_diff, fpr_diff, di)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.0353 0.0 -0.0511 1.113
0.045 0.125 0.001 0.8854
0.0264 0.1 0.0323 1.0821
